In [1]:
pip install langchain transformers sentence-transformers faiss-cpu pypdf


In [2]:
from pypdf import PdfReader

reader = PdfReader("Meta’s Q1 2024 Financial Report.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])


Meta Reports First Quarter 2024 Results
MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter 
ended March 31, 2024.
"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is 
another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making 
steady progress building the metaverse as well."
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% ChangeIn millions, except percentages and per share amounts 2024 2023
Revenue $ 36,455 $ 28,645  27 %
Costs and expenses  22,637  21,418  6 %
Income from operations $ 13,818 $ 7,227  91 %
Operating margin  38 %  25 %
Provision for income taxes $ 1,814 $ 1,598  14 %
Effective tax rate  13 %  22 %
Net income $ 12,369 $ 5,709  117 %
Diluted earnings per share (EPS) $ 4.71 $ 2.20  114 %
First Quarter 2024 Operational and Other Financial Highligh

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(text)


In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [5]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(embeddings[0].shape[0])
index.add(embeddings.cpu().numpy())

In [6]:
query = "What was Meta’s revenue in Q1 2024?"
query_embedding = model.encode([query])
D, I = index.search(np.array(query_embedding), k=3)
top_chunks = [chunks[i] for i in I[0]]


In [7]:
from transformers import pipeline

qa_pipeline = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", max_new_tokens=200)

prompt = f"""Based on the following context: {top_chunks}

Answer the query: {query}"""

result = qa_pipeline(prompt)
print(result[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Based on the following context: ['steady progress building the metaverse as well."\nFirst Quarter 2024 Financial Highlights\nThree Months Ended March 31,\n% ChangeIn millions, except percentages and per share amounts 2024 2023\nRevenue $ 36,455 $ 28,645  27 %\nCosts and expenses  22,637  21,418  6 %\nIncome from operations $ 13,818 $ 7,227  91 %\nOperating margin  38 %  25 %\nProvision for income taxes $ 1,814 $ 1,598  14 %\nEffective tax rate  13 %  22 %\nNet income $ 12,369 $ 5,709  117 %', 'Meta Reports First Quarter 2024 Results\nMENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter \nended March 31, 2024.\n"It\'s been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is \nanother step towards building the world\'s leading AI. We\'re seeing healthy growth across our apps and we continue making \nsteady progress building the metaverse as well."', 'the "R

In [8]:
query = "What was Meta’s revenue in Q1 2024?"
query_embedding = model.encode([query])
D, I = index.search(np.array(query_embedding), k=3)
top_chunks = [chunks[i] for i in I[0]]

prompt = f"""Based on the following context: {top_chunks}

Answer the query: {query}"""

result = qa_pipeline(prompt)
print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Based on the following context: ['steady progress building the metaverse as well."\nFirst Quarter 2024 Financial Highlights\nThree Months Ended March 31,\n% ChangeIn millions, except percentages and per share amounts 2024 2023\nRevenue $ 36,455 $ 28,645  27 %\nCosts and expenses  22,637  21,418  6 %\nIncome from operations $ 13,818 $ 7,227  91 %\nOperating margin  38 %  25 %\nProvision for income taxes $ 1,814 $ 1,598  14 %\nEffective tax rate  13 %  22 %\nNet income $ 12,369 $ 5,709  117 %', 'Meta Reports First Quarter 2024 Results\nMENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter \nended March 31, 2024.\n"It\'s been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is \nanother step towards building the world\'s leading AI. We\'re seeing healthy growth across our apps and we continue making \nsteady progress building the metaverse as well."', 'the "R

In [9]:
query = "What were the key financial highlights for Meta in Q1 2024?"
query_embedding = model.encode([query])
D, I = index.search(np.array(query_embedding), k=5)
top_chunks = [chunks[i] for i in I[0]]

prompt = f"""Based on the following context: {top_chunks}

Answer the query: {query}"""

result = qa_pipeline(prompt)
print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Based on the following context: ['Meta Reports First Quarter 2024 Results\nMENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter \nended March 31, 2024.\n"It\'s been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is \nanother step towards building the world\'s leading AI. We\'re seeing healthy growth across our apps and we continue making \nsteady progress building the metaverse as well."', 'steady progress building the metaverse as well."\nFirst Quarter 2024 Financial Highlights\nThree Months Ended March 31,\n% ChangeIn millions, except percentages and per share amounts 2024 2023\nRevenue $ 36,455 $ 28,645  27 %\nCosts and expenses  22,637  21,418  6 %\nIncome from operations $ 13,818 $ 7,227  91 %\nOperating margin  38 %  25 %\nProvision for income taxes $ 1,814 $ 1,598  14 %\nEffective tax rate  13 %  22 %\nNet income $ 12,369 $ 5,709  117 %', "Absent

**STEP 2**

In [10]:
!pip install camelot-py[cv]

import camelot
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.8.0
    Uninstalling pypdf-5.8.0:
      Successfully uninstalled pypdf-5.8.0


In [11]:
tables = camelot.read_pdf("Meta’s Q1 2024 Financial Report.pdf", pages="all", flavor="stream")

/usr/local/lib/python3.11/dist-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (38.6, 611.0668939999999, 579.48, 764.3955603636364)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/usr/local/lib/python3.11/dist-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (38.6, 466.906924, 579.4999999999999, 789.3610116470587)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/usr/local/lib/python3.11/dist-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (38.6, 406.906924, 579.4899999999999, 789.3049892380952)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/usr/local/lib/python3.11/dist-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (38.59999999999996, 58.90692399999999, 579.5, 789.1710606666666)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/usr/local/lib/python3.1

In [12]:
for i, t in enumerate(tables):
    print(i, t.shape, t.df.head())

0 (7, 1)                                                    0
0            Meta Reports First Quarter 2024 Results
1  MENLO PARK, Calif. – April 24, 2024 – Meta Pla...
2                              ended March 31, 2024.
3  "It's been a good start to the year," said Mar...
4  another  step  towards  building  the  world's...
1 (11, 6)                                                    0  1       2  \
0            First Quarter 2024 Financial Highlights              
1                                                                 
2  In millions, except percentages and per share ...       2024   
3                                            Revenue  $  36,455   
4                                 Costs and expenses     22,637   

                              3       4         5  
0                                                  
1  Three Months Ended March 31,                    
2                                  2023  % Change  
3                             $  28,645      27 %  


In [19]:
income_df = tables[1].df

In [22]:

income_df.columns = (
    income_df.iloc[2] # The actual header seems to be in the 3rd row (index 2)
    .astype(str) # Convert to string in case of non-string types
    .str.strip()                 # remove leading/trailing spaces
    .str.replace(r"\n", " ", regex=True)  # replace any internal newlines
)


income_df = income_df.iloc[3:].reset_index(drop=True)



income_df.columns = ["Metric", "2024 Value", "2024", "2023 Value", "2023", "% Change"]



for col in ["2024", "2023"]:   # columns "2024" and "2023" which contain the percentage change
    income_df[col] = (
        income_df[col].astype(str) # Convert to string explicitly
        .str.replace(r"[^0-9%.+-]", "", regex=True) # Keep numbers, %, ., +, -
        .str.replace("%","", regex=True)
    )
    income_df[col] = pd.to_numeric(income_df[col], errors='coerce')



for col in ["2024 Value", "2023 Value"]:
    income_df[col] = (
        income_df[col].astype(str) # Convert to string explicitly
        .str.replace(r"[^0-9.]", "", regex=True) # Keep only numbers and '.'
    )
    income_df[col] = pd.to_numeric(income_df[col], errors='coerce')


display(income_df.head())

,Metric,2024 Value,2024,2023 Value,2023,% Change
0,Net income,NaN,12369.00,NaN,5709.0,117 %
1,Diluted earnings per share (EPS),NaN,4.71,NaN,2.2,114 %


In [29]:
exp_df = tables[6].df

In [40]:
# Inspect what your raw column names are after selecting the correct table
print("Original columns after initial table extraction:")
print(exp_df.columns.tolist())

# Drop the header rows (index 0 and 1)
exp_df = exp_df.iloc[2:].reset_index(drop=True)

# Inspect columns after dropping header rows
print("\nColumns after dropping header rows:")
print(exp_df.columns.tolist())


# Based on the structure of tables[6].df and the head after dropping rows,
# the relevant columns appear to be at specific indices.
# Metric is likely the first column (index 0)
# 2024 values are likely at index 3
# 2023 values are likely at index 5

# Rename the relevant columns using their indices
exp_df = exp_df.rename(columns={
    exp_df.columns[0]: "Metric",
    exp_df.columns[3]: "2024",
    exp_df.columns[5]: "2023"
})

# Inspect columns after renaming
print("\nColumns after renaming:")
print(exp_df.columns.tolist())

# Now convert the '2024' and '2023' columns to floats safely
for col in ["2024", "2023"]:
    # Check if the column exists before trying to clean and convert
    if col in exp_df.columns:
        exp_df[col] = (
            exp_df[col].astype(str) # Convert to string explicitly
                  .str.replace(r"[^0-9.\-]", "", regex=True)  # remove any stray symbols except digits, '.', and '-'
        )
        exp_df[col] = pd.to_numeric(exp_df[col], errors='coerce')
    else:
        print(f"Warning: Column '{col}' not found in DataFrame.")


# Check your cleaned DataFrame
display(exp_df.head())

Original columns after initial table extraction:
['Metric', '', '', '2564.0', '', 'nan', '9,381']

Columns after dropping header rows:
['Metric', '', '', '2564.0', '', 'nan', '9,381']

Columns after renaming:
['Metric', '', '', '2024', '', '2023', '9,381']


,Metric,,,2024,,2023,"9,381"
0,Income from operations,,,13818.0,,NaN,"7,227"
1,"Interest and other income, net",,,365.0,,NaN,80
2,Income before provision for income taxes,,,14183.0,,NaN,"7,307"
3,Provision for income taxes,,,1814.0,,NaN,"1,598"
4,Net income,$,,12369.0,$,NaN,"5,709"


In [31]:
import numpy as np

def hybrid_query(query, k=3):
    # 1) Textual retrieval
    q_emb = model.encode([query])
    D, I = index.search(np.array(q_emb), k=k)
    text_context = "\n\n".join([chunks[i] for i in I[0]])

    # 2) Structured lookup
    q_lower = query.lower()
    if "net income" in q_lower:
        row = income_df[income_df["Metric"].str.contains("Net income", case=False)].iloc[0]
        structured = {
            "Metric":"Net income",
            "2024":row["2024"],
            "2023":row["2023"],
        }
    elif "operating expenses" in q_lower:
        # filter expense rows
        rows = exp_df[exp_df["Metric"].isin([
            "Cost of revenue",
            "Research and development",
            "Marketing and sales",
            "General and administrative"
        ])]
        structured = rows.set_index("Metric")[["2024"]].to_dict()["2024"]
    else:
        structured = {}

    # 3) Combined prompt
    prompt = f"""
Text context:
{text_context}

Structured data:
{structured}

Answer the query:
{query}
"""
    out = qa_pipeline(prompt, max_new_tokens=200)[0]["generated_text"]
    return out



In [35]:
print(hybrid_query("What was Meta’s net income in Q1 2024 compared to Q1 2023?"))


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Text context:
steady progress building the metaverse as well."
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% ChangeIn millions, except percentages and per share amounts 2024 2023
Revenue $ 36,455 $ 28,645  27 %
Costs and expenses  22,637  21,418  6 %
Income from operations $ 13,818 $ 7,227  91 %
Operating margin  38 %  25 %
Provision for income taxes $ 1,814 $ 1,598  14 %
Effective tax rate  13 %  22 %
Net income $ 12,369 $ 5,709  117 %

Meta Reports First Quarter 2024 Results
MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter 
ended March 31, 2024.
"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is 
another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making 
steady progress building the metaverse as well."

the "Reconciliation of GAAP to Non-GAAP Results

In [41]:
print(hybrid_query("Summarize Meta’s operating expenses in Q1 2024."))


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Text context:
steady progress building the metaverse as well."
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% ChangeIn millions, except percentages and per share amounts 2024 2023
Revenue $ 36,455 $ 28,645  27 %
Costs and expenses  22,637  21,418  6 %
Income from operations $ 13,818 $ 7,227  91 %
Operating margin  38 %  25 %
Provision for income taxes $ 1,814 $ 1,598  14 %
Effective tax rate  13 %  22 %
Net income $ 12,369 $ 5,709  117 %

(In millions)
(Unaudited)
Three Months Ended March 31,
2024 2023
Cash flows from operating activities
Net income $ 12,369 $ 5,709 
Adjustments to reconcile net income to net cash provided by operating activities:
Depreciation and amortization  3,374  2,524 
Share-based compensation  3,562  3,051 
Deferred income taxes  (456)  (620) 
Impairment charges for facilities consolidation, net  240  770 
Other  (66)  (7) 
Changes in assets and liabilities:
Accounts receivable  2,520  2,546

the "Reconciliation of GAAP to Non-GAAP Resul

**STEP 3**

In [42]:
from transformers import pipeline

In [43]:
# 1) Load a text2text model (fast)
rewrite_pipe = pipeline("text2text-generation", model="google/flan-t5-small", device=0)

def optimize_query(raw_q: str) -> str:
    prompt = (
        "Rewrite this question to be more precise for information retrieval:\n"
        f"Question: {raw_q}\n"
        "Optimized:"
    )
    out = rewrite_pipe(prompt, max_new_tokens=32)[0]["generated_text"]
    # strip any prefix like "Optimized:"
    return out.replace("Optimized:", "").strip()

# Example
raw = "Tell me net income compare to last year"
opt_q = optimize_query(raw)
print("Optimized query:", opt_q)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Optimized query: What is the average net income for the year 2000?


In [46]:
from sentence_transformers import CrossEncoder
import numpy as np

# Load a lightweight cross‑encoder
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", device="cuda")

def retrieve_and_rerank(query: str, chunks: list, k: int = 10, rerank_top: int = 3):
    # 1) retrieve top-k by vector search
    q_emb = model.encode([query]) # Corrected: use 'model' instead of 'embedder'
    D, I = index.search(np.array(q_emb), k=k)
    candidates = [chunks[i] for i in I[0]]

    # 2) score with cross-encoder
    scores = reranker.predict([[query, c] for c in candidates])

    # 3) pick top-rerank_top
    top_idxs = np.argsort(scores)[::-1][:rerank_top]
    return [candidates[i] for i in top_idxs]

# Example
opt_q = optimize_query("What was Meta’s operating margin in Q1 2024?")
best_chunks = retrieve_and_rerank(opt_q, chunks)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

for size in [400, 600, 800]:
    splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=100)
    test_chunks = splitter.split_text(text)
    # rebuild FAISS index per experiment...


In [48]:
import numpy as np

def precision_at_k(retrieved_ids, relevant_ids, k):
    return len(set(retrieved_ids[:k]) & set(relevant_ids)) / k

def recall_at_k(retrieved_ids, relevant_ids, k):
    return len(set(retrieved_ids[:k]) & set(relevant_ids)) / len(relevant_ids)

def reciprocal_rank(retrieved_ids, relevant_ids):
    for rank, idx in enumerate(retrieved_ids, start=1):
        if idx in relevant_ids:
            return 1.0 / rank
    return 0.0

def mrr(all_retrieved, all_relevant):
    return np.mean([reciprocal_rank(r, rel) for r, rel in zip(all_retrieved, all_relevant)])


In [55]:
queries = [
    "What was Meta’s net income in Q1 2024 compared to Q1 2023?",
    "Summarize Meta’s operating expenses in Q1 2024."
]

refs = [
    "Net income in Q1 2024 was $12,369 million versus $5,709 million in Q1 2023, an increase of roughly 116.6%.",
    "In Q1 2024, Meta’s operating expenses were: Cost of revenue $6,640 M; R&D $9,978 M; Marketing & sales $2,564 M; G&A $3,455 M; Total $22,637 M."
]


In [56]:
preds = []
for q in queries:
    answer = hybrid_query(q)     # or whatever function you wrote
    preds.append(answer.strip()) # strip whitespace for consistency

# Inspect predictions
for q, p, r in zip(queries, preds, refs):
    print("Q:", q)
    print("P:", p)
    print("R:", r)
    print("---")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Q: What was Meta’s net income in Q1 2024 compared to Q1 2023?
P: Text context:
steady progress building the metaverse as well."
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% ChangeIn millions, except percentages and per share amounts 2024 2023
Revenue $ 36,455 $ 28,645  27 %
Costs and expenses  22,637  21,418  6 %
Income from operations $ 13,818 $ 7,227  91 %
Operating margin  38 %  25 %
Provision for income taxes $ 1,814 $ 1,598  14 %
Effective tax rate  13 %  22 %
Net income $ 12,369 $ 5,709  117 %

Meta Reports First Quarter 2024 Results
MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter 
ended March 31, 2024.
"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is 
another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making 
steady progress building the meta

In [57]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

results_bleu  = bleu.compute(predictions=preds, references=refs)
results_rouge = rouge.compute(predictions=preds, references=refs)

print("BLEU:", results_bleu)
print("ROUGE:", results_rouge)


BLEU: {'bleu': 0.011922435581623854, 'precisions': [0.058011049723756904, 0.018005540166204988, 0.006944444444444444, 0.002785515320334262], 'brevity_penalty': 1.0, 'length_ratio': 10.969696969696969, 'translation_length': 724, 'reference_length': 66}
ROUGE: {'rouge1': np.float64(0.11599763872491145), 'rouge2': np.float64(0.06514221570437964), 'rougeL': np.float64(0.10074773711137347), 'rougeLsum': np.float64(0.10950413223140497)}


In [62]:
# This cell expects these objects to already exist:
#   model, index, chunks, income_df, exp_df, qa_pipeline, hybrid_query

import pandas as pd
from IPython.display import display

# 15 diverse test queries
test_queries = [
    "What was Meta’s revenue in Q1 2024?",
    "How did Q1 2024 net income compare to Q1 2023?",
    "What was the tax rate, and how much did provision for taxes increase?",
    "How many daily active people did Meta report?",
    "What is the difference between GAAP and non‑GAAP revenue figures?",
    "What guidance did Meta give for Q2 2024 revenue?",
    "What were Meta’s capital expenditures in Q1 2024 and how did they compare to Q1 2023?",
    "Summarize Meta’s free cash flow in Q1 2024.",
    "How did Meta’s operating margin change year‑over‑year in Q1 2024?",
    "What was Meta’s diluted earnings per share in Q1 2024?",
    "Compare Meta’s advertising revenue between Q1 2024 and Q1 2023.",
    "What were Meta’s total costs and expenses in Q1 2024?",
    "How much did Meta spend on research and development in Q1 2024?",
    "What was the effective tax rate for Meta in Q1 2024?",
    "Summarize the segment revenue for Reality Labs in Q1 2024."
]

# sanity‑check that hybrid_query exists
if 'hybrid_query' not in globals():
    raise RuntimeError("Please run the Step 2 cell that defines hybrid_query before this cell.")

# run each query through your pipeline
results = []
for q in test_queries:
    ans = hybrid_query(q)
    results.append({"Query": q, "Answer": ans})

# make a DataFrame and display it
df_results = pd.DataFrame(results)
display(df_results)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

,Query,Answer
0,What was Meta’s revenue in Q1 2024?,\nText context:\nsteady progress building the ...
1,How did Q1 2024 net income compare to Q1 2023?,\nText context:\nAbsent any changes to our tax...
2,"What was the tax rate, and how much did provis...",\nText context:\nAbsent any changes to our tax...
3,How many daily active people did Meta report?,\nText context:\nMeta Reports First Quarter 20...
4,What is the difference between GAAP and non‑GA...,\nText context:\nNon-GAAP Financial Measures \...
5,What guidance did Meta give for Q2 2024 revenue?,\nText context:\nsteady progress building the ...
6,What were Meta’s capital expenditures in Q1 20...,\nText context:\nsteady progress building the ...
7,Summarize Meta’s free cash flow in Q1 2024.,\nText context:\nsteady progress building the ...
8,How did Meta’s operating margin change year‑ov...,\nText context:\nMeta Reports First Quarter 20...
9,What was Meta’s diluted earnings per share in ...,\nText context:\nsteady progress building the ...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_2['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)